based on https://huggingface.co/ShinyQ/indobert-sentiment-analysis-indonesian-university-reviews

In [1]:
import os, math, random, csv
from dataclasses import dataclass
from typing import List, Dict

import re
import string
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from tqdm import tqdm
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    get_linear_schedule_with_warmup,
)
from sklearn.metrics import classification_report, f1_score


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [26]:


# --- Model checkpoint from Hugging Face ---
MODEL_ID = "indobenchmark/indobert-large-p2"

# --- Define 5 labels (adjust names to your dataset!)
labels = ['anger', 'happy', 'sadness', 'love', 'fear']
label2id: Dict[str, int] = {lb: i for i, lb in enumerate(labels)}
id2label: Dict[int, str] = {i: lb for lb, i in label2id.items()}

# --- Reproducibility ---
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [27]:
@dataclass
class Config:
    csv_path: str = "Twitter_Emotion_Dataset.csv"   # single file with text + label
    epochs: int = 10
    batch_size: int = 16
    lr: float = 5e-5
    max_len: int = 250
    weight_decay: float = 0.01
    warmup_ratio: float = 0.06
    grad_clip: float = 1.0
    num_workers: int = 2
    val_ratio: float = 0.2          # 80% train / 20% val
    out_dir: str = "../ckpt_unireviews_5labels"

cfg = Config()

In [28]:
# Indonesian stopwords
stop_words = stopwords.words("indonesian")


In [29]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    # text = str(text).lower()
    text = " ".join([y for y in text.split() if y not in stop_words])
    return text


In [38]:


class CSVDataset(Dataset):
    def __init__(self, csv_path: str, tokenizer, max_len: int):
        self.rows = []
        with open(csv_path, newline="", encoding="utf-8") as f:
            reader = csv.DictReader(f)
            for r in reader:
                t = clean_text((r.get("tweet") or "").strip())
                y = (r.get("label") or "").strip().lower()
                if t and y in label2id:
                    self.rows.append((t, label2id[y]))
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self): return len(self.rows)

    def __getitem__(self, idx):
        text, y = self.rows[idx]
        enc = self.tokenizer(
            text,
            truncation=True,
            max_length=self.max_len,
            padding=False,
            return_tensors="pt",
        )
        item = {k: v.squeeze(0) for k, v in enc.items()}
        item["labels"] = torch.tensor(y, dtype=torch.long)
        return item

In [39]:
def collate_fn(batch: List[Dict[str, torch.Tensor]]):
    keys = ["input_ids", "attention_mask"]
    if "token_type_ids" in batch[0]:
        keys.append("token_type_ids")
    max_len = max(x["input_ids"].size(0) for x in batch)
    out = {}
    for k in keys:
        pad_id = tokenizer.pad_token_id if k == "input_ids" else 0
        out[k] = torch.stack([
            torch.nn.functional.pad(x[k], (0, max_len - x[k].size(0)), value=pad_id)
            for x in batch
        ])
    out["labels"] = torch.stack([x["labels"] for x in batch])
    return out


In [40]:
def evaluate(model, loader, device):
    model.eval()
    gold, pred = [], []
    with torch.no_grad():
        for batch in loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            logits = model(**{k: v for k, v in batch.items() if k != "labels"}).logits
            pred.extend(logits.argmax(-1).cpu().tolist())
            gold.extend(batch["labels"].cpu().tolist())
    print(classification_report(gold, pred, target_names=labels, digits=4))


In [41]:
    device = "cuda" if torch.cuda.is_available() else "cpu"
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

    # --- Load dataset & bt_split into train/val ---
    full_ds = CSVDataset(cfg.csv_path, tokenizer, cfg.max_len)
    n_val = int(len(full_ds) * cfg.val_ratio)
    n_train = len(full_ds) - n_val
    train_ds, val_ds = random_split(full_ds, [n_train, n_val])

In [42]:

train_loader = DataLoader(train_ds, batch_size=cfg.batch_size,
                          shuffle=True, num_workers=cfg.num_workers,
                          collate_fn=collate_fn)
val_loader = DataLoader(val_ds, batch_size=cfg.batch_size,
                        shuffle=False, num_workers=cfg.num_workers,
                        collate_fn=collate_fn)

In [43]:
# --- Model ---
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_ID, num_labels=len(labels)
)
model.config.id2label = id2label
model.config.label2id = label2id
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-large-p2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1

In [44]:
# --- Optimizer & scheduler ---
no_decay = ["bias", "LayerNorm.weight"]
grouped = [
    {"params": [p for n,p in model.named_parameters() if not any(nd in n for nd in no_decay)],
      "weight_decay": cfg.weight_decay},
    {"params": [p for n,p in model.named_parameters() if any(nd in n for nd in no_decay)],
      "weight_decay": 0.0},
]
optimizer = torch.optim.AdamW(grouped, lr=cfg.lr)
total_steps = cfg.epochs * math.ceil(len(train_loader))
warmup_steps = int(cfg.warmup_ratio * total_steps)
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps
)

In [45]:
    # --- Training loop ---
    os.makedirs(cfg.out_dir, exist_ok=True)
    for epoch in range(1, cfg.epochs + 1):
        model.train()
        running = 0.0
        for step, batch in tqdm(enumerate(train_loader, 1)):
            batch = {k: v.to(device) for k, v in batch.items()}
            out = model(**{k: v for k, v in batch.items() if k != "labels"},
                        labels=batch["labels"])
            loss = out.loss
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), cfg.grad_clip)
            optimizer.step(); scheduler.step(); optimizer.zero_grad()
            running += loss.item()
            if step % 50 == 0:
                print(f"epoch {epoch} step {step}: loss {running/50:.4f}")
                running = 0.0

        print(f"\n=== Validation after epoch {epoch} ===")
        evaluate(model, val_loader, device)

        save_path = os.path.join(cfg.out_dir, f"epoch{epoch}")
        model.save_pretrained(save_path)
        tokenizer.save_pretrained(save_path)

    print("\nTraining done.")

50it [00:32,  1.63it/s]

epoch 1 step 50: loss 1.5654


100it [01:04,  1.60it/s]

epoch 1 step 100: loss 1.0323


150it [01:36,  1.60it/s]

...

200it [02:06,  1.69it/s]

epoch 10 step 200: loss 0.0071


221it [02:19,  1.58it/s]


=== Validation after epoch 10 ===


              precision    recall  f1-score   support

       anger     0.7902    0.7826    0.7864       207
       happy     0.7876    0.8091    0.7982       220
     sadness     0.6716    0.6995    0.6853       193
        love     0.8154    0.8346    0.8249       127
        fear     0.8644    0.7669    0.8127       133

    accuracy                         0.7761       880
   macro avg     0.7859    0.7785    0.7815       880
weighted avg     0.7784    0.7761    0.7767       880


Training done.
